In [1]:
import os
import sys
import re
import pandas as pd
import zipfile
from enum import Enum, EnumMeta
from io import StringIO
from Tables import Tables
from Spells import Spells
from Items import Items
from IPython.display import display
from random import sample

In [131]:
class BaseOps:
    
    @staticmethod
    def bytes2Num(barray, signed=False, order='little'):
        return int.from_bytes(barray, signed=signed, byteorder=order)
    
    @staticmethod
    def num2Bytes(num, length, signed=False, order='little'):
        return num.to_bytes(length, signed=signed, byteorder=order)
    

class DataPiece(BaseOps):
    
    def __init__(self, position, length):
        self.position = position
        self.length = length
        self.end = self.position + self.length
        self.signed = False
        self.order = 'little'
    
    def getBytes(self, buffer, seg_offset):
        return buffer[seg_offset + self.position:
                      seg_offset + self.end]
    
    def getValue(self, buffer, seg_offset):
        return self.bytes2Num(buffer[seg_offset + self.position:
                                     seg_offset + self.end],
                              self.signed,
                              self.order)
    
    def setValue(self, buffer, seg_offset, new_value):
        if not isinstance(new_value, int):
            new_value = new_value.value
        buffer[seg_offset + self.position: 
               seg_offset + self.end] = self.num2Bytes(new_value,
                                                       self.length,
                                                       self.signed,
                                                       self.order)    
        
    def incValue(self, buffer, seg_offset, inc_value):
        self.setValue(buffer, 
                      seg_offset,
                      self.getValue(buffer, seg_offset) + inc_value)
        

class BaseTable(DataPiece, Enum):
    
    def __init__(self, *params):  # Python does not have to invoke superclass __init__()
        _len = len(params)
        self.position = params[0]
        self.length = params[1]
        self.end = self.position + self.length
        self.signed = params[2] if _len > 2 else False
        self.order = params[3] if _len > 3 else 'little'
        self.table = params[4] if _len > 4 else None
        self.optimum = params[5] if _len > 5 else None
    

    def getRepr(self, buffer, seg_offset, parent=Tables):
        _value = self.getValue(buffer, seg_offset)
        if self.table:
            try:
                return getattr(parent, self.table)(_value).name
            except ValueError:
                return '0x{0:08X}'.format(_value)               
        else:
            return str(_value)
        
    def optimize(self, buffer, seg_offset):
        if self.optimum is not None:
            self.setValue(buffer, seg_offset, self.optimum)  

In [132]:
class Segments(BaseTable):
    
    EXP = (0x0018, 4, False, 'little', None, 30000000)
    Gold = (0x001c, 4, False, 'little', None, 10000)
    States = (0x0020, 4, False, 'little', 'State', None)
    MaxHP = (0x0024, 2, False, 'little', None, 200)
    CurrentHP = (0x0026, 2, False, 'little', None, 200)
    Reputation = (0x0044, 1, True, 'little', None, 20)
    HideInShadow = (0x0045, 1, False, 'little', None, 100)
    NatureAC = (0x0046, 2, True, 'little', None, -10)
    EffectiveAC = (0x0048, 2, True, 'little', None, -10)
    CrushAC = (0x004A, 2, True, 'little', None, -10)
    MissileAC = (0x004C, 2, True, 'little', None, -10)
    PiercingAC = (0x004E, 2, True, 'little', None, -10)
    SlashingAC = (0x0050, 2, True, 'little', None, -10)
    THAC0 = (0x0052, 1, False, 'little', None, 0)
    NumOfAttack = (0x0053, 1, False, 'little', None, 10)
    SaveVsDeath = (0x0054, 1, False, 'little', None, 0)
    SaveVsWands = (0x0055, 1, False, 'little', None, 0)
    SaveVsPolymorph = (0x0056, 1, False, 'little', None, 0)
    SaveVsBreath = (0x0057, 1, False, 'little', None, 0)
    SaveVsSpell = (0x0058, 1, False, 'little', None, 0)
    ResistFire = (0x0059, 1, False, 'little', None, 100)
    ResistCold = (0x005A, 1, False, 'little', None, 100)
    ResistElectricity = (0x005B, 1, False, 'little', None, 100)
    ResistAcid = (0x005C, 1, False, 'little', None, 100)
    ResistMagic = (0x005D, 1, False, 'little', None, 100)
    ResistMagicFire = (0x005E, 1, False, 'little', None, 100)
    ResistMagicCold = (0x005F, 1, False, 'little', None, 100)
    ResistSlashing = (0x0060, 1, False, 'little', None, 100)
    ResistCrushing = (0x0061, 1, False, 'little', None, 100)
    ResistPiercing = (0x0062, 1, False, 'little', None, 100)
    ResistMissile = (0x0063, 1, False, 'little', None, 100)
    DetectIllusion = (0x0064, 1, False, 'little', None, 100)
    SetTraps = (0x0065, 1, False, 'little', None, 100)
    Lore = (0x0066, 1, False,'little',  None, 100)
    Lockpicking = (0x0067, 1, False, 'little', None, 100)
    Stealth = (0x0068, 1,False, 'little',  None, 100)
    FindTraps = (0x0069, 1, False, 'little',None, 100)
    PickPockets = (0x006A, 1, False, 'little', None, 100)
    Fatigue = (0x006B, 1,False, 'little',  None, 0)
    Intoxication = (0x006C, 1, False, 'little', None, 0)
    Luck = (0x006D, 1, False, 'little', None,  100)
    #LSwordPro = (0x006E, 1, False, 'little', None, 5)
    #SSwordPro = (0x006F, 1, False, 'little', None, 5)
    #BowPro = (0x0070, 1, False, 'little', None, 5)
    #SpearPro = (0x0071, 1, False, 'little', None, 5)
    #BluntPro = (0x0072, 1, False, 'little', None, 5)
    #SpikedPro = (0x0073, 1, False, 'little', None, 5)
    #AxePro = (0x0074, 1, False, 'little', None, 5)
    #MissilePro = (0x0075, 1, False, 'little', None, 5)
    #UnusedPro1 = (0x0076, 1, False, 'little', None, 5)
    #UnusedPro2 = (0x0077, 1, False, 'little', None, 5)
    #UnusedPro3 = (0x0078, 1, False, 'little', None, 5)
    #UnusedPro4 = (0x0079, 1, False, 'little', None, 5)
    #UnusedPro5 = (0x007a, 1, False, 'little', None, 5)
    #UnusedPro6 = (0x007b, 1, False, 'little', None, 5)
    #UnusedPro7 = (0x007c, 1, False, 'little', None, 5)
    #NightMareMode = (0x007d, 1, False, 'little', None, None)
    #Translucency = (0x007e, 1, False, 'little', None, None)
    #MurderInc = (0x007f, 1, False, 'little', None, None)
    #ReputationJoin = (0x0080, 1, False, 'little', None, None)
    #ReputationLeave = (0x0081, 1, False, 'little', None, None)
    TurnUndead = (0x0082, 1, False, 'little', None, 100)
    Tracking = (0x0083, 1, False, 'little', None, 100)
    LvClass1 = (0x0234, 1, False, 'little', None, 1)
    LvClass2 = (0x0235, 1, False, 'little', None, 1)
    LvClass3 = (0x0236, 1, False, 'little', None, 1)
    Sex = (0x0237, 1, False, 'little', 'Gender', None)
    Strength = (0x0238, 1, False, 'little', None, 25)
    StrBonus = (0x0239, 1, False, 'little', None, 0)
    Intelligence = (0x023A, 1, False, 'little', None, 25)
    Wisdom = (0x023B, 1, False, 'little', None, 25)
    Dexterity = (0x023C, 1, False, 'little', None, 25)
    Constitution = (0x023D, 1, False, 'little', None, 25)
    Charisma = (0x023E, 1, False, 'little', None, 25)
    Morale = (0x023F, 1, False, 'little', None, 100)
    MoraleBreak = (0x0240, 1, False, 'little', None, 0)
    RacialEnemy = (0x0241, 1, False, 'little', 'Race', None)
    MoraleRecovery = (0x0242, 2, False, 'little', None, 0)
    Kit = (0x0244, 4, False, 'little', 'Kit', None)
    Race = (0x0272, 1, False, 'little', 'Race', None)
    Class = (0x0273, 1, False, 'little', 'Class', None)
    Gender = (0x0275, 1, False, 'little', 'Gender', None)
    Alignment = (0x027B, 1, False, 'little', 'Alignment', None)
    KnownSpellOffset = (0x02a0, 4, False, 'little', None, None)
    KnownSpellCount = (0x02a4, 4, False, 'little', None, None)
    MemSpellOffset = (0x02a8, 4, False, 'little', None, None)
    MemSpellCount = (0x02ac, 4, False, 'little', None, None)
    MemorizedSpellOffset = (0x02b0, 4, False, 'little', None, None)
    MemorizedSpellCount = (0x02b4, 4, False, 'little', None, None)
    ItemSlotOffset = (0x02b8, 4, False, 'little', None, None)
    ItemOffset = (0x02bc, 4, False, 'little', None, None)
    ItemCount = (0x02c0, 4, False, 'little', None, None)
    EffectOffset = (0x02c4, 4, False, 'little', None, None)
    EffectCount = (0x02c8, 4, False, 'little', None, None)
    DialogRef = (0x02cc, 8, False, 'little', None, None)

In [133]:
class BaseSeg(BaseOps):
    
    def __init__(self, filename, name, buffer, parent, extra_internal_offset, offset_ref, length_ref, count_ref):
        self.filename = filename
        self.name = name
        self.buffer = buffer
        self.parent = parent
        self.extra_internal_offset = extra_internal_offset
        self.childrens = list() 
        self.next_sibling = None
        self.offset_ref = offset_ref
        self.length_ref = length_ref
        self.count_ref = count_ref
    
    
    @property
    def _start(self):
        if self.offset_ref:
            return self.offset_ref.getValue(buffer=self.buffer, 
                                            seg_offset=self.parent.adj_offset_buffer)
        else:
            return 0       
        
    @property
    def adj_offset_segment(self):
        return self._start + self.extra_internal_offset

    
    @property
    def adj_offset_buffer(self): 
        if self.parent:
            return self.parent.adj_offset_buffer + self.adj_offset_segment
        else:
            return self.adj_offset_segment      
    
    
    @property
    def length(self):
        if self.length_ref:
            return self.length_ref.getValue(buffer=self.buffer, 
                                            seg_offset=self.parent.adj_offset_buffer)
        else:
            return None
    
    @property
    def count(self):
        if self.count_ref:
            return self.count_ref.getValue(buffer=self.buffer, 
                                           seg_offset=self.parent.adj_offset_buffer)
        else:
            return None
    
    
    def expand(self, volume):
        if self.length_ref:
            self.length_ref.incValue(buffer=self.buffer, seg_offset=self.parent.adj_offset_buffer, inc_value=volume)
        if self.next_sibling:
            self.next_sibling.move(volume)
        if self.parent:
            self.parent.expand(volume)
        
    def move(self, volume):
        self.offset_ref.incValue(buffer=self.buffer, seg_offset=self.parent.adj_offset_buffer, inc_value=volume)
        if self.next_sibling:
            self.next_sibling.move(volume)     
            
    def flush(self):
        with open(self.filename, 'wb+') as _file:
            _file.write(self.buffer)

In [134]:
class CharSeg(BaseSeg):
    
    EXTRA_INTERNAL_OFFSET = 10

    def __init__(self, filename, name, buffer, parent, extra_internal_offset, offset_ref, length_ref):
        super().__init__(filename, name, buffer, parent, extra_internal_offset, offset_ref, length_ref, None)
        self.data = pd.DataFrame()
        self.read()
        self.readAppendix()
        
    def read(self):
        self.data = pd.DataFrame()
        for _name, _item in Segments.__members__.items():
            self.data = self.data.append({'Item': _name,
                                          'File_offset': _item.position + self.adj_offset_buffer,
                                          'CRE_offset': _item.position,
                                          'Length': _item.length,
                                          'Number': _item.getValue(self.buffer, self.adj_offset_buffer),
                                          'Value': _item.getRepr(self.buffer, self.adj_offset_buffer)},
                                        ignore_index=True)
        self.data['File_offset'] = self.data['File_offset'].apply(lambda x: '0x{0:06X}h'.format(int(x)))
        self.data['CRE_offset'] = self.data['CRE_offset'].apply(lambda x: '0x{0:06X}h'.format(int(x)))
        self.data['Length'] = self.data['Length'].astype(int)
        self.data['Number'] = self.data['Number'].apply(lambda x:str(int(x)))
        self.data.set_index('Item', inplace=True)
        self.data.index.name = None
        
    def optimize(self):
        for _name, _item in Segments.__members__.items():
            _item.optimize(self.buffer, self.adj_offset_buffer)
        self.effects.optimize()
        
            
    def setValue(self, item, new_value):
        if not isinstance(new_value, int):
            new_value = new_value.value       
        getattr(Segments, item.name).setValue(self.buffer, self.adj_offset_buffer, new_value)   
        
    def readAppendix(self):
        self.knownSpell = KnownSpell(self.filename, 'KnownSpell', self.buffer, self,
                                     DataPiece(Segments.KnownSpellOffset.position, 
                                               Segments.KnownSpellOffset.length),
                                     DataPiece(Segments.KnownSpellCount.position, 
                                               Segments.KnownSpellCount.length),
                                     12)
        self.memSpell = MemSpell(self.filename, 'MemSpell', self.buffer, self,
                                 DataPiece(Segments.MemSpellOffset.position, 
                                           Segments.MemSpellOffset.length),
                                 DataPiece(Segments.MemSpellCount.position, 
                                           Segments.MemSpellCount.length),
                                 16)
        self.memorizedSpell = MemorizedSpell(self.filename, 'MemorizedSpell', self.buffer, self, 
                                             DataPiece(Segments.MemorizedSpellOffset.position, 
                                                       Segments.MemorizedSpellOffset.length),
                                             DataPiece(Segments.MemorizedSpellCount.position, 
                                                       Segments.MemorizedSpellCount.length),
                                             12)
        self.itemSlots = ItemSlots(self.filename, 'ItemSlots', self.buffer, self,
                                   DataPiece(Segments.ItemSlotOffset.position, 
                                             Segments.ItemSlotOffset.length),
                                   None,
                                   2, 80)
        self.inventoryItems = InventoryItems(self.filename, 'InventoryItems', self.buffer, self, 
                           DataPiece(Segments.ItemOffset.position, 
                                     Segments.ItemOffset.length),
                           DataPiece(Segments.ItemCount.position, 
                                     Segments.ItemCount.length),
                           20)
        self.effects = Effects(self.filename, 'Effects', self.buffer, self, 
                               DataPiece(Segments.EffectOffset.position, 
                                         Segments.EffectOffset.length),
                               DataPiece(Segments.EffectCount.position, 
                                         Segments.EffectCount.length),
                               264)
        self.knownSpell.next_sibling = self.memSpell
        self.memSpell.next_sibling = self.memorizedSpell
        self.memorizedSpell.next_sibling = self.itemSlots
        self.itemSlots.next_sibling = self.inventoryItems
        self.inventoryItems.next_sibling = self.effects
        
    @property
    def weapons(self):
        return self.effects.weapons
    
    @property
    def items(self):
        return self.inventoryItems.items
    
    @property
    def spells(self):
        return self.knownSpell.spells

In [135]:
class AppendixSeg(BaseSeg):
    
    class Record(BaseTable):
        pass
    
    def __init__(self, filename, name, buffer, parent, offset_ref, count_ref, record_length, segment_length=None):
        super().__init__(filename, name, buffer, parent, 0, offset_ref, None, count_ref)
        self.record_length = record_length
        self.data = pd.DataFrame()
        self.records = list()
        self.read()
        
    def read(self):
        self.data = pd.DataFrame()
        self.records = list()
        if self.count:
            for _i in range(self.count):
                _start = self.adj_offset_buffer + _i * self.record_length
                _end = _start + self.record_length
                self.records.append((_start, _end))
                _data = dict([(_name, _item.getRepr(self.buffer, 
                                                    _start)) for _name, _item in self.Record.__members__.items()])
                self.data = self.data.append(_data, ignore_index=True)
                
    def delete(self, index):
        if index < self.count:
            self.buffer[self.records[index][0]: self.records[index][1]] = []
            self.read()
            if self.count:
                self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, -1)
            self.expand(-self.record_length)
    
    def insert(self, template=0):
        if template < self.count:
            self.buffer[self.records[0][0]: self:records[0][0]] = self.buffer[self.records[index][0]: 
                                                                              self:records[index][1]]
            self.read()
            if self.count:
                self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, 1)
            self.expand(self.record_length) 
            
    def setValue(self, item, value, index=0):
        _start = self.records[index][0]
        if not isinstance(new_value, int):
            value = value.value
        getattr(self.Record, item.name).setValue(self.buffer, _start, value)
        
    
    @staticmethod
    def searchName(_value, table):
        _value = _value.replace('\x00', '')
        for _cls in table.__dict__.values():
            if isinstance(_cls, EnumMeta):
                for _item in _cls:
                    if _item.value == _value:
                        return _item.name, _cls.__name__
        else:
            return _value, None

In [136]:
class MemSpell(AppendixSeg):
    pass

class MemorizedSpell(AppendixSeg):
    pass


In [137]:
class ItemSlots(AppendixSeg):
    
    class Position(BaseTable):
        
        HELMET = (0x0000, 2)
        ARMOR = (0x0002, 2)
        SHIELD = (0x0004, 2)
        GLOVES = (0x0006, 2)
        RINGS = (0x0008, 4)
        AMULET = (0x000c, 2)
        BELT = (0x000e, 2)
        BOOTS = (0x0010, 2)
        WEAPONS = (0x0012, 4)
        EXTRAWEAPONS = (0x0016, 4)
        QUIVERS = (0x001a, 6)
        EXTRAQUIVER = (0x0020, 2)
        CLOAK = (0x0022, 2)
        QUICK = (0x0024, 6)
        INVENTORY = (0x002a, 32) 
        MAGICWEAPON = (0x004a, 2)
        SELECTEDWEAPON = (0x004c, 2)
        SELECTEDABILITY = (0x004e, 2)
    
    def delete(self, index):
        _start = self.adj_offset_buffer + index * self.record_length
        _end = _start + self.record_length
        self.buffer[_start: _end] = b'\xff\xff'
    
    def place(self, inventory):
        
        self.helmet = bytearray(), 1
        self.armor = bytearray(), 1
        self.shield = bytearray(), 1
        self.gloves = bytearray(), 1
        self.rings = bytearray(), 2
        self.amulet = bytearray(), 1
        self.belt = bytearray(), 1
        self.boots = bytearray(), 1
        self.weapons = bytearray(), 2
        self.extraweapons = bytearray(), 2
        self.quivers = bytearray(), 3
        self.extraquiver = bytearray(), 2
        self.cloak = bytearray(), 1
        self.quick = bytearray(), 3
        self.inventory = bytearray(), 16
        self.magicweapon = bytearray(), 1
        self.selectedweapon = bytearray(), 1
        self.selectedability = bytearray(), 1
        
        occupied = { 'CLOAKS': self.cloak,
                     'ARROWS': self.quivers,
                     'TWO_HANDED_SWORDS': self.weapons,
                     'DROW_ITEMS': self.inventory,
                     'QUIVERS': self.quivers,
                     'POTIONS': self.quick,
                     'Exceptions': self.inventory,
                     'DAGGERS': self.weapons,
                     'WANDS': self.quick,
                     'RINGS': self.rings,
                     'CROSSBOWS': self.weapons,
                     'PLOT_ITEMS': self.inventory,
                     'HAMMERS': self.weapons,
                     'SHIELDS': self.shield,
                     'SECRET_PANTALOON_STUFF': self.inventory,
                     'STAVES': self.weapons,
                     'SPEARS': self.weapons,
                     'ROBES': self.armor,
                     'CHAINMAILS': self.armor,
                     'BOWS': self.weapons,
                     'HALBERDS': self.weapons,
                     'BOLTS': self.quivers,
                     'HELMS': self.helmet,
                     'DRAGON_SCALES': self.inventory,
                     'IWD': self.inventory,
                     'AXES': self.weapons,
                     'BOOKS': self.inventory,
                     'SLINGS': self.weapons,
                     'KEYS': self.inventory,
                     'BELTS': self.belt,
                     'BOOTS': self.boots,
                     'TOMES': self.inventory,
                     'SCROLLS': self.inventory,
                     'BRACERS': self.gloves,
                     'BAGS': self.inventory,
                     'ONE_HANDED_SWORDS': self.weapons,
                     'AMULETS': self.amulet,
                     'BLUNT_WEAPONS': self.weapons,
                     'BULLETS': self.quivers,
                     'DARTS': self.quivers,
                     'RODS': self.quick,
                     'FULL_PLATE_ARMORS': self.armor,
                     'FAMILIARS': self.inventory,
                     'MISC': self.inventory,
                     'COMPONENTS': self.inventory,
                     'LEATHER_ARMORS': self.armor          
                    }
        
        for _index, _category in enumerate(list(zip(*inventory))[1]):
            _code = self.num2Bytes(_index, 2)
            if _category in occupied.keys():
                if len(occupied[_category][0]) < occupied[_category][1] * 2:
                    occupied[_category][0].extend(_code)
                else:
                    for _other in [self.inventory, self.quick, self.quivers, self.shield, self.helmet, 
                                   self.armor, self.gloves, self.rings, self.amulet, self.belt, self.boots,
                                   self.weapons, self.cloak, self.extraweapons]:
                        if len(_other[0]) < _other[1] * 2:
                            _other[0].extend(_code)
                            break
            else:
                self.inventory[0].extend(_code)             
        for _item in self.Position:
            _code = getattr(self, _item.name.lower())
            _bytes = _code[0]
            _bytes.extend(b'\xff' * (_code[1] * 2 - len(_code[0])))
            _start = self.adj_offset_buffer + _item.value[0]
            _end = _start + _item.value[1]
            self.buffer[_start: _end] = _bytes
            #_item.setValue(self.buffer, self.adj_offset_buffer, self.bytes2Num(_bytes))

In [138]:
class KnownSpell(AppendixSeg):

    
    class Record(BaseTable):

        SPELL = (0x0000, 8)
        LEVEL = (0x0008, 2)
        TYPE = (0x000a, 2)
        
    SPELL_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    SPELL_PATTERN = re.compile(r'(?P<type>\w{4})(?P<level>\d)(?P<id>\d+)')
    SPELL_TYPE = {'SPWI': 1,
                  'SPCL': 0,
                  'SPIN': 2,
                  'SPPR': 3,
                  1: 'Wizard',
                  0: 'Cleric',
                  2:' Innate',
                  3: 'Special'}
    
    @property
    def end(self):
        return self.adj_offset_buffer + self.count * self.record_length
    
    def read(self):
        self.data = pd.DataFrame(columns=self.Record.__members__.keys())
        self.records = [[], [], [], []]
        for index in range(self.count):
            _start = self.adj_offset_buffer + index * self.record_length
            _end = _start + self.record_length
            _record = self.buffer[_start:_end]
            _spell, _cls = self.searchName(self.Record.SPELL.getBytes(_record, 0).decode('UTF-8'),
                                           table=Spells)
            _type = self.Record.TYPE.getValue(_record, 0) 
            _level = self.Record.LEVEL.getValue(_record, 0) + 1
            self.records[_type].append(_record)
            self.data = self.data.append({'SPELL': _spell,
                                          'TYPE': self.SPELL_TYPE[_type],
                                          'LEVEL': _level},
                                        ignore_index=True)
            self.data['LEVEL'] = self.data['LEVEL'].astype(int)
        
        
    
    def prepareTemplate(self, spell):
        _string = re.match(self.SPELL_PATTERN, spell.value)
        _template = self.SPELL_TEMPLATE[:]
        _spell = self.bytes2Num(bytearray(spell.value, encoding='UTF-8'))
        _level = int(_string.group('level')) - 1
        if _level < 0:
            raise('Wait to figure out Lv.10 spells!')
        _type = self.SPELL_TYPE[_string.group('type')]
        self.Record.SPELL.setValue(_template, 0, _spell)
        self.Record.LEVEL.setValue(_template, 0, _level)
        self.Record.TYPE.setValue(_template, 0, _type)
        return _template, _type, _level
    
    
    def _delete(self, _spell):
        _count = 0
        _template, _type, _level = self.prepareTemplate(_spell)
        for _i, _r in self.records[_type]:
            if _r == _template:
                self.records[_type].pop(_i)
                _count += 1
        return _count
    
    def _insert(self, _spell):
        _count = 0
        _template, _type, _level = self.prepareTemplate(_spell)
        if not _template in self.records[_type]:
            self.records[_type].append(_template)
            _count += 1
        return _count       
    
    def delete(self, spell):
        self.read()
        _count = 0
        if isinstance(spell, EnumMeta):
            _count = 0
            for _item, _spell in spell.__members__.items():
                _count += self._delete(_spell)
        else:
            _count = self._delete(spell)
        if _count != 0:
            self.buffer[self.adj_offset_buffer : self.end] = self.merge(self.records)
            self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, -_count)
            self.expand(-self.record_length * _count)
            self.read()
        
    def insert(self, spell):
        _count = 0
        self.read()
        if isinstance(spell, EnumMeta):
            _count = 0
            for _item, _spell in spell.__members__.items():
                _count += self._insert(_spell)
        else:
            _count = self._insert(spell)
        self.buffer[self.adj_offset_buffer : self.end] = self.merge(self.records)
        self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, _count)
        self.expand(self.record_length * _count)
        self.read()
                
    
    def merge(self, spells):
        _array = bytearray()
        for _type in self.records:
            for _record in sorted(_type, key=lambda x: self.Record.LEVEL.getValue(x, 0)):
                _array.extend(_record)
        return _array
    
            
    @property
    def spells(self):
        self.read()
        return self.data

In [139]:
class InventoryItems(AppendixSeg):
    
    GOODITEMS =  [Items.AMULETS.Amulet_of_Power, 
                  Items.AMULETS.Amulet_of_the_Master_Harper,
                  Items.AXES.Axe_of_the_Unyielding_PLUS5,
                  Items.BELTS.Belt_of_Inertial_Barrier,
                  Items.BLUNT_WEAPONS.Flail_of_Ages_PLUS5,
                  Items.BOOTS.Boots_of_Speed,
                  Items.BOWS.Short_Bow_of_Gesen,
                  Items.BOWS.Taralash_PLUS5,
                  Items.BRACERS.Gauntlets_of_Extraordinary_Specialization,
                  Items.CHAINMAILS.Aslyferund_Elven_Chain_PLUS5,
                  Items.CLOAKS.Cloak_of_Mirroring,
                  Items.CLOAKS.Cloak_of_Bravery,
                  Items.ROBES.Robe_of_the_Good_Archmagi,
                  Items.ROBES.Robe_of_the_Neutral_Archmagi,
                  Items.CROSSBOWS.Firetooth_PLUS5,
                  Items.DAGGERS.Dagger_of_the_Star_PLUS5,
                  Items.HALBERDS.Ravager_PLUS6,
                  Items.HAMMERS.Runehammer_PLUS5,
                  Items.HELMS.Circlet_of_Netheril_1,
                  Items.HELMS.Kiels_Helmet,
                  Items.HELMS.Helm_of_Charm_Protection,
                  Items.FULL_PLATE_ARMORS.Enkidus_Full_Plate_PLUS3,
                  Items.RINGS.Ring_of_Wizardry,
                  Items.RINGS.Ring_of_Holiness,
                  Items.RINGS.Ring_of_Gaxx,
                  Items.RINGS.Ring_of_Acuity,
                  Items.RINGS.Heartwood_Ring,
                  Items.RINGS.Ring_of_AntiVenom,
                  Items.SHIELDS.Reflection_Shield_PLUS1,
                  Items.SHIELDS.Shield_of_Harmony_PLUS2,
                  Items.SLINGS.Erinne_Sling_PLUS5,
                  Items.SPEARS.Spear_of_the_Unicorn_PLUS2,
                  Items.STAVES.Staff_of_the_Magi,
                  Items.STAVES.Staff_of_the_Ram_PLUS6,
                  Items.ONE_HANDED_SWORDS.Blackrazor_Long_Sword_PLUS3,
                  Items.ONE_HANDED_SWORDS.Scimitar_of_Speed_PLUS2_Belm,
                  Items.ONE_HANDED_SWORDS.Adjatha_the_Drinker_Long_Sword_PLUS2,
                  Items.ONE_HANDED_SWORDS.Celestial_Fury_Katana_PLUS3,
                  Items.ONE_HANDED_SWORDS.Short_Sword_of_Mask_PLUS5,
                  Items.ONE_HANDED_SWORDS.Angurvadal_PLUS5,
                  Items.ONE_HANDED_SWORDS.The_Answerer_PLUS4,
                  Items.ONE_HANDED_SWORDS.Scimitar_PLUS5_Defender,
                  Items.TWO_HANDED_SWORDS.Psions_Blade_PLUS5,
                  Items.QUIVERS.Bag_of_Plenty_PLUS2,
                  Items.QUIVERS.Case_of_Plenty_PLUS2,
                  Items.QUIVERS.Quiver_of_Plenty_PLUS2]
    
    GOODITEMS = [Items.RINGS.Ring_of_Wizardry,
                 Items.RINGS.Ring_of_Acuity,
                 Items.RINGS.Ring_of_Holiness,
                 Items.HELMS.Helm_of_Balduran,
                 Items.HELMS.Helm_of_Charm_Protection,
                 Items.HELMS.Kiels_Helmet,
                 Items.HELMS.Pale_Green_Ioun_Stone,
                 Items.BRACERS.Gauntlets_of_Weapon_Expertise,
                 Items.BOOTS.Boots_of_Speed,
                 Items.AMULETS.Amulet_of_Power,
                 Items.SHIELDS.Large_Shield_PLUS1_PLUS4_vs_Missiles,
                 #Items.SHIELDS.Kiels_Buckler,
                 Items.FULL_PLATE_ARMORS.The_Magma_Bulwark_PLUS2,
                 Items.CHAINMAILS.Elven_Chain_Mail,
                 Items.ROBES.Robe_of_the_Good_Archmagi,
                 Items.ROBES.Robe_of_the_Neutral_Archmagi,
                 Items.SLINGS.Sling_of_UNERRING_ACCURACY,
                 Items.CROSSBOWS.Heavy_Crossbow_of_Accuracy,
                 Items.CROSSBOWS.Light_Crossbow_of_Speed,
                 Items.BOWS.Long_Bow_of_Marksmanship,
                 Items.ONE_HANDED_SWORDS.Scimitar_PLUS2,
                 Items.STAVES.Staff_Spear_PLUS2,
                 Items.BLUNT_WEAPONS.The_Stupefier_PLUS1,
                 Items.ONE_HANDED_SWORDS.Long_Sword_PLUS1_Flame_Tongue,
                 #Items.DAGGERS.Boomerang_Dagger_PLUS2,
                 Items.ONE_HANDED_SWORDS.Bastard_Sword_PLUS1_Albruin,
                 Items.AXES.Klogarath_PLUS4,
                 Items.QUIVERS.Bag_of_Plenty_PLUS1,
                 Items.QUIVERS.Case_of_Plenty_PLUS1,
                 Items.QUIVERS.Quiver_of_Plenty_PLUS1,
                 Items.BAGS.Bag_of_Holding,
                 Items.BELTS.Belt_of_Inertial_Barrier,
                 Items.CLOAKS.Cloak_of_Bravery,
                 Items.CLOAKS.Cloak_of_NonDetection]
    
    class Record(BaseTable):
        NAME = (0x0000, 8)
        EXPIRATION = (0x0008, 1)
        ELAPSED = (0x0009, 1)
        QUANTITY_1 = (0x000a, 2)
        QUANTITY_2 = (0x000c, 2)
        QUANTITY_3 = (0x000e, 2)
        IDENTIFIED = (0x000f, 1)
        UNSTEALABLE = (0x0010, 1)
        STOLEN = (0x0011, 1)
        UNDROPPABLE = (0x0012, 1)
        
    MAX_COUNT = 40
    
    ITEM_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

    @property
    def end(self):
        return self.adj_offset_buffer + self.count * self.record_length
    
    def read(self):
        self.inventory = list()
        self.records = list()
        self.data = pd.DataFrame(columns=list(self.Record.__members__.keys()) + ['CATEGORY'])
        for index in range(self.count):
            _start = self.adj_offset_buffer + index * self.record_length
            _end = _start + self.record_length
            _record = self.buffer[_start:_end]
            self.records.append((_start, _end))
            _dict = dict()
            for _item in self.Record:
                if _item == self.Record.NAME:
                    _dict[self.Record.NAME.name], _dict['CATEGORY'] = self.searchName(_item.getBytes(_record, 0).decode('UTF-8'),
                                                                                      table=Items)
                else:
                    _dict[_item.name] = _item.getValue(_record, 0)
            self.inventory.append((_dict[self.Record.NAME.name], _dict['CATEGORY']))
            self.data = self.data.append(_dict, ignore_index=True)
        for col in self.data.columns:
            if not col in [self.Record.NAME.name, 'CATEGORY']:
                self.data[col] = self.data[col].astype(int)
                
    def _insert(self, item, quality=1):
        if self.inventory == []:
            _template = self.ITEM_TEMPLATE[:]
            self.Record.NAME.setValue(_template, 0, self.bytes2Num(bytearray(item.value, encoding='UTF-8')))
            self.Record.QUANTITY_1.setValue(_template, 0, quality)            
            if self.count < self.MAX_COUNT: 
                self.buffer[self.adj_offset_buffer: self.adj_offset_buffer] = _template
                self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, 1)
                self.expand(self.record_length)
            else:
                self.buffer[self.adj_offset_buffer:self.end] = self.buffer[self.adj_offset_buffer:
                                                                           self.end - self.record_length]
                self.buffer[self.adj_offset_buffer: self.adj_offset_buffer] = _template
        elif not item.value in list(zip(*self.inventory))[0]:
            _template = self.ITEM_TEMPLATE[:]
            self.Record.NAME.setValue(_template, 0, self.bytes2Num(bytearray(item.value, encoding='UTF-8')))
            self.Record.QUANTITY_1.setValue(_template, 0, quality)            
            if self.count < self.MAX_COUNT: 
                self.buffer[self.records[0][0]:self.records[0][0]] = _template
                self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, 1)
                self.expand(self.record_length)
            else:
                self.buffer[self.adj_offset_buffer:self.end] = self.buffer[self.adj_offset_buffer:
                                                                           self.end - self.record_length]
                self.buffer[self.records[0][0]:self.records[0][0]] = _template
                

    def insert(self, item, quality=1):
        self.read()
        self._insert(item, quality)
        self.read()
        self.parent.itemSlots.place(self.inventory)
        
        
    def delete(self, item):
        self.read()
        _index = 0
        for _item, _pos in zip(list(zip(*self.inventory))[0], self.records):
            if item.value == _item:
                self.buffer[_pos[0]: _pos[1]] = []
                self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, -1)
                self.expand(-self.record_length)
                self.parent.itemSlots.delete(_index)
            _index += 1
        self.read()
        self.parent.itemSlots.delete(_index)
        
    @property
    def items(self):
        self.read()
        return self.data
    
    def optimize(self, size=None):
        if size is None:
            size = self.MAX_COUNT
        if size > len(self.GOODITEMS):
            size = len(self.GOODITEMS)
        self.read()
        for _item in sample(self.GOODITEMS, size):
            self._insert(_item) 
        self.read()
        self.parent.itemSlots.place(self.inventory)

In [140]:
class Effects(AppendixSeg):
    
    class WeaponOptima(Enum):
        BastardSword = 5
        LongSword = 5
        ShortSword = 5
        Axe = 5
        TwoHandedSword = 5
        Katana = 5
        ScimitarWakizashiNinjaTo = 5
        Dagger = 5
        WarHammer = 5
        Spear = 5
        Halberd = 5
        FlailMorningstar = 5
        Mace = 5
        QuarterStaff = 5
        Crossbow = 5
        LongBow = 5
        ShortBow = 5
        Darts = 5
        Sling = 5
        #Blackjack = 5
        #Gun = 5
        #MartialArts = 5
        TwoHandedWeaponSkill = 2
        SwordandShieldSkill = 2
        SingleWeaponSkill = 2
        TwoWeaponSkill = 3
        Club = 5
        #ExtraProficiency2 = 5
        #ExtraProficiency3 = 5
        #ExtraProficiency4 = 5
        #ExtraProficiency5 = 5
        #ExtraProficiency6 = 5
        #ExtraProficiency7 = 5
        #ExtraProficiency8 = 5
        #ExtraProficiency9 = 5
        #ExtraProficiency10 = 5
        #ExtraProficiency11 = 5
        #ExtraProficiency12 = 5
        #ExtraProficiency13 = 5
        #ExtraProficiency14 = 5
        #ExtraProficiency15 = 5
        #ExtraProficiency16 = 5
        #ExtraProficiency17 = 5
        #ExtraProficiency18 = 5
        #ExtraProficiency19 = 5
        #ExtraProficiency20 = 5
    
    WEAPON_OPCODE = 0xe9
    WEAPON_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 233, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 95, 
                                 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 255, 
                                 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    
    class Record(BaseTable):
        OPCODE = (0x0008, 4)
        TARGET = (0x000c, 4, False, 'little', 'Target')
        POWER = (0x0010, 4)
        PARAM_1 = (0x0014, 4, False, 'little', None, 5)
        PARAM_2 = (0x0018, 4, False, 'little', 'WeaponType')
        TIME = (0x001c, 4, False, 'little', 'Time')
        
    def insert(self, template=None):
        self.read()
        if isinstance(template, int) and template < self.count:
            self.buffer[self.records[0][0]: self.records[0][0]] = self.buffer[self.records[index][0]: 
                                                                              self:records[index][1]]
        elif template is None:
            self.buffer[self.records[0][0]: self.records[0][0]] = bytearray(self.WEAPON_TEMPLATE)
        elif isinstance(template, bytearray):
            self.buffer[self.records[0][0]: self.records[0][0]] = template[:]
        if self.count:
            self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, 1)
        self.expand(self.record_length) 
        self.read()
        
    @property
    def weapons(self):
        self.read()
        return self.data[self.data['OPCODE'] == str(self.WEAPON_OPCODE)]
    
    def _getWeaponOptimum(self, code):
        _weapon = Tables.WeaponType(code).name
        return getattr(self.WeaponOptima, _weapon).value
    
    def optimize(self, more_weapon=True):
        weapons = list()
        self.read()
        for _start, _end in self.records:
            if self.Record.OPCODE.getValue(self.buffer, _start) == self.WEAPON_OPCODE:
                _code = self.Record.PARAM_2.getValue(self.buffer, _start)
                weapons.append(_code)
                self.Record.PARAM_1.setValue(self.buffer, _start, self._getWeaponOptimum(_code))
        if more_weapon:
            for _weapon, _code in Tables.WeaponType.__members__.items():
                if not _code.value in weapons:
                    _template = self.WEAPON_TEMPLATE[:]
                    self.Record.PARAM_2.setValue(_template, 0, _code)
                    self.Record.PARAM_1.setValue(_template, 0, self._getWeaponOptimum(_code)) 
                    self.insert(_template)
        self.read()

In [194]:
class CRE():
    
    class Blocks(BaseTable):
        NAME = (0x0008, 32)
        CRE_OFFSET = (0x0028, 4)
        CRE_LENGTH = (0x002c, 4)
        
    
    def __init__(self, filename):
        self.filename = filename
        self.pwd = os.path.dirname(self.filename)
        self.file = os.path.basename(self.filename)
        self.zipfile = zipfile.ZipFile(self.filename)
        self.file_list = self.zipfile.namelist()
        self.chr_buffer = bytearray(self.zipfile.read(self.file_list[1]))
        self.adj_offset_buffer = 0
        self.read()
        
    @property
    def name(self):
        _name = self.chr_buffer[self.Blocks.NAME.position:
                                self.Blocks.NAME.position + self.Blocks.NAME.length]
        while _name[-1] == 0:
            _name = _name[:-1]
        return _name.decode('UTF-8')
    
    @property
    def weapons(self):
        return self.cre.weapons
    
    @property
    def spells(self):
        return self.cre.spells
    
    @property
    def items(self):
        return self.cre.items
    
    @property
    def data(self):
        _df = self.cre.data
        return _df[~_df.index.str.contains('Ref|Count|Offset')]

    def expand(self, volume):
        pass
    
    def move(self, volume):
        pass
    
    def read(self):
        self.cre = CharSeg(self.filename,
                           self.name,
                           self.chr_buffer,
                           self,
                           0,
                           DataPiece(self.Blocks.CRE_OFFSET.position,
                                     self.Blocks.CRE_OFFSET.length),
                           DataPiece(self.Blocks.CRE_LENGTH.position,
                                     self.Blocks.CRE_LENGTH.length)
                           )
        
    def optimize(self):
        self.cre.optimize()
        
    def setValue(self, item, new_value):
        self.cre.setValue(item, new_value)
    
    def addSpell(self, spell):
        self.cre.knownSpell.insert(spell)
        
    def removeSpell(self, spell):
        self.cre.knownSpell.delete(spell)
        
    def giveItem(self, item, quantity=1):
        self.cre.inventoryItems.insert(item, quantity)
        
    def removeItem(self, item):
        self.cre.inventoryItems.delete(item)
        
    def goodItem(self, size=34):
        self.cre.inventoryItems.optimize(size)
        
    def pack(self):
        bio_buffer = self.zipfile.read(self.file_list[0])
        self.zipfile.close()
        with zipfile.ZipFile(self.filename, 'w') as target:
            for _f, _name in zip([bio_buffer, self.chr_buffer], self.file_list):
                target.writestr(_name, _f)
                
    def export(self):
        _col = columns=['name','record_length',
                       'offset_pos', 'offset_value', 
                       'count_pos', 'count_value',
                       'total_length']
        self.summary_df = pd.DataFrame(columns=_col)
        with open(self.file_list[1], 'wb') as target:
            target.write(self.chr_buffer)
        
        for _name, _seg in [ ('knownspell', self.cre.knownSpell),
                             ('memspell', self.cre.memSpell),
                             ('memorizedspell', self.cre.memorizedSpell),
                             ('itemslots', self.cre.itemSlots),
                             ('items', self.cre.inventoryItems),
                             ('effects', self.cre.effects)]:
            _record_length = getattr(_seg, 'record_length', None)
            _offset_res = getattr(_seg, 'offset_ref', None)
            if _offset_res:
                _offset_pos = '0x{0:06X}'.format(_offset_res.position)
                _offset_value = _offset_res.getValue(_seg.buffer, _seg.parent.adj_offset_buffer)
            else:
                _offset_value = None
            _count_res = getattr(_seg, 'count_ref', None)
            if _count_res:
                _count_pos = '0x{0:06X}'.format(_count_res.position)
                _count_value = _count_res.getValue(_seg.buffer, _seg.parent.adj_offset_buffer)
            elif _name == 'itemslots':
                _count_value = 40
            _length = getattr(_seg, 'length', None)
            if _length is None:
                _length = _count_value * _record_length
            self.summary_df = self.summary_df.append(dict(zip(_col, [_name, _record_length, 
                                                                     _offset_pos, _offset_value,
                                                                     _count_pos, _count_value,
                                                                     _length])),
                                                    ignore_index=True)
        for col in ['record_length', 'offset_value', 'count_value', 'total_length']:
            self.summary_df[col] = self.summary_df[col].astype(int)
        return self.summary_df
                
            

In [208]:
filename = [r'01FIGHT.bg1character', r'02THIEF.bg1character', r'03MAGE.bg1character', r'04PALAD.bg1character']
a = [CRE(_) for _ in filename]

In [209]:
a[0].export()

,name,record_length,offset_pos,offset_value,count_pos,count_value,total_length
0,knownspell,12,0x0002A0,0,0x0002A4,0,0
1,memspell,16,0x0002A8,724,0x0002AC,16,256
2,memorizedspell,12,0x0002B0,0,0x0002B4,0,0
3,itemslots,2,0x0002B8,980,0x0002B4,40,80
4,items,20,0x0002BC,1060,0x0002C0,1,20
5,effects,264,0x0002C4,1080,0x0002C8,5,1320


In [210]:
a[1].export()

,name,record_length,offset_pos,offset_value,count_pos,count_value,total_length
0,knownspell,12,0x0002A0,724,0x0002A4,2,24
1,memspell,16,0x0002A8,748,0x0002AC,16,256
2,memorizedspell,12,0x0002B0,1004,0x0002B4,6,72
3,itemslots,2,0x0002B8,1076,0x0002B4,40,80
4,items,20,0x0002BC,1156,0x0002C0,1,20
5,effects,264,0x0002C4,1176,0x0002C8,1,264


In [211]:
a[2].export()

,name,record_length,offset_pos,offset_value,count_pos,count_value,total_length
0,knownspell,12,0x0002A0,724,0x0002A4,4,48
1,memspell,16,0x0002A8,772,0x0002AC,17,272
2,memorizedspell,12,0x0002B0,1044,0x0002B4,7,84
3,itemslots,2,0x0002B8,1128,0x0002B4,40,80
4,items,20,0x0002BC,0,0x0002C0,0,0
5,effects,264,0x0002C4,1208,0x0002C8,36,9504


In [212]:
a[3].export()

,name,record_length,offset_pos,offset_value,count_pos,count_value,total_length
0,knownspell,12,0x0002A0,724,0x0002A4,3,36
1,memspell,16,0x0002A8,760,0x0002AC,16,256
2,memorizedspell,12,0x0002B0,1016,0x0002B4,9,108
3,itemslots,2,0x0002B8,1124,0x0002B4,40,80
4,items,20,0x0002BC,1204,0x0002C0,2,40
5,effects,264,0x0002C4,1244,0x0002C8,3,792


In [104]:
#[_.optimize() for _ in a]

[None, None, None, None]

In [80]:
a[0].setValue(Segments.Class, Tables.Class.FIGHTER_DRUID)
a[0].setValue(Segments.Kit, Tables.Kit.TRUECLASS)
a[1].setValue(Segments.Class, Tables.Class.CLERIC_RANGER)
a[1].setValue(Segments.Kit, Tables.Kit.GODHELM)
a[2].setValue(Segments.Class, Tables.Class.FIGHTER_MAGE_THIEF)
a[2].setValue(Segments.Kit, Tables.Kit.SHADOWDANCER)
a[3].setValue(Segments.Class, Tables.Class.FIGHTER_MAGE_THIEF)
a[3].setValue(Segments.Kit, Tables.Kit.SHADOWDANCER)

In [117]:
a[0].addSpell(Spells.Magic_CLERIC)
a[1].addSpell(Spells.Magic_CLERIC)
a[2].addSpell(Spells.Magic_WIZARD)
a[3].addSpell(Spells.Magic_WIZARD)

KeyboardInterrupt: 

In [112]:
[_.goodItem() for _ in a]

[None, None, None, None]

In [40]:
#[_.pack() for _ in a]

[None, None, None, None]

In [118]:
a[0].spells

KeyboardInterrupt: 

In [114]:
a[1].items

,NAME,EXPIRATION,ELAPSED,QUANTITY_1,QUANTITY_2,QUANTITY_3,IDENTIFIED,UNSTEALABLE,STOLEN,UNDROPPABLE,CATEGORY
0,Quiver_of_Plenty_PLUS1,0,0,1,0,0,0,1,0,0,QUIVERS
1,Boots_of_Speed,0,0,1,0,0,0,1,0,0,BOOTS
2,Helm_of_Balduran,0,0,1,0,0,0,1,0,0,HELMS
3,Bastard_Sword_PLUS1_Albruin,0,0,1,0,0,0,1,0,0,ONE_HANDED_SWORDS
4,Staff_Spear_PLUS2,0,0,1,0,0,0,1,0,0,STAVES
5,Klogarath_PLUS4,0,0,1,0,0,0,1,0,0,AXES
6,Elven_Chain_Mail,0,0,1,0,0,0,1,0,0,CHAINMAILS
7,Large_Shield_PLUS1_PLUS4_vs_Missiles,0,0,1,0,0,0,1,0,0,SHIELDS
8,Case_of_Plenty_PLUS1,0,0,1,0,0,0,1,0,0,QUIVERS
9,Ring_of_Wizardry,0,0,1,0,0,0,1,0,0,RINGS


In [95]:
a[1].weapons

,OPCODE,PARAM_1,PARAM_2,POWER,TARGET,TIME
0,233,3,TwoWeaponSkill,0,NONE,PERMANENT_AFTER_DEATH
1,233,2,SingleWeaponSkill,0,NONE,PERMANENT_AFTER_DEATH
2,233,2,SwordandShieldSkill,0,NONE,PERMANENT_AFTER_DEATH
3,233,2,TwoHandedWeaponSkill,0,NONE,PERMANENT_AFTER_DEATH
4,233,5,Sling,0,NONE,PERMANENT_AFTER_DEATH
5,233,5,Darts,0,NONE,PERMANENT_AFTER_DEATH
6,233,5,ShortBow,0,NONE,PERMANENT_AFTER_DEATH
7,233,5,LongBow,0,NONE,PERMANENT_AFTER_DEATH
8,233,5,Crossbow,0,NONE,PERMANENT_AFTER_DEATH
9,233,5,QuarterStaff,0,NONE,PERMANENT_AFTER_DEATH


In [48]:
filename = [r'05CLERIC.bg1character', r'06RANGER.bg1character', r'07BARB.bg1character', r'08BARD.bg1character']
a = [CRE(_) for _ in filename]

In [45]:
[_.goodItem() for _ in a]

[None, None, None, None]

In [46]:
[_.pack() for _ in a]

[None, None, None, None]

In [49]:
a[0].weapons

,OPCODE,PARAM_1,PARAM_2,POWER,TARGET,TIME


In [97]:
f1 = CRE(r'01FIGHT.bg1character')

FileNotFoundError: [Errno 2] No such file or directory: '01FIGHT.bg1character'

In [78]:
f1.weapons

,OPCODE,PARAM_1,PARAM_2,POWER,TARGET,TIME
0,233,3,TwoWeaponSkill,0,NONE,PERMANENT_AFTER_DEATH
1,233,2,SingleWeaponSkill,0,NONE,PERMANENT_AFTER_DEATH
2,233,2,SwordandShieldSkill,0,NONE,PERMANENT_AFTER_DEATH
3,233,2,TwoHandedWeaponSkill,0,NONE,PERMANENT_AFTER_DEATH
4,233,5,Sling,0,NONE,PERMANENT_AFTER_DEATH
5,233,5,Darts,0,NONE,PERMANENT_AFTER_DEATH
6,233,5,ShortBow,0,NONE,PERMANENT_AFTER_DEATH
7,233,5,LongBow,0,NONE,PERMANENT_AFTER_DEATH
8,233,5,Crossbow,0,NONE,PERMANENT_AFTER_DEATH
9,233,5,QuarterStaff,0,NONE,PERMANENT_AFTER_DEATH


In [79]:
f1.goodItem()

In [87]:
f1.items.count()

NAME           162
EXPIRATION     162
ELAPSED        162
QUANTITY_1     162
QUANTITY_2     162
QUANTITY_3     162
IDENTIFIED     162
UNSTEALABLE    162
STOLEN         162
UNDROPPABLE    162
CATEGORY       162
dtype: int64

In [88]:
f1.goodItem()

In [89]:
f1.items.count()

NAME           162
EXPIRATION     162
ELAPSED        162
QUANTITY_1     162
QUANTITY_2     162
QUANTITY_3     162
IDENTIFIED     162
UNSTEALABLE    162
STOLEN         162
UNDROPPABLE    162
CATEGORY       162
dtype: int64

In [92]:
f1.removeSpell(Spells.Magic_CLERIC)

NameError: name 'instance' is not defined

In [213]:
[b'\xff\xff'] * 2

[b'\xff\xff', b'\xff\xff']

In [215]:
a = []
a.extend([b'\xff\xff'] * 2)

In [216]:
a

[b'\xff\xff', b'\xff\xff']